In [8]:
class Loginer:
    
    def __init__(self, email, password, csrf):
        self.email = email
        self.password = password
        self.csrf = csrf
    
    def login(self, req):
        url = 'https://www.worldquantvrc.com/login/process'
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) snap Chromium/71.0.3578.98 Chrome/71.0.3578.98 Safari/537.36'}
        sim_data = dict(EmailAddress = self.email, Password = self.password, _xsrf = self.csrf)
        login = req.post(url, data = sim_data, verify = False, allow_redirects = True, headers = headers)
        try:
            dct = login.json()
            return dct['status']
        except json.decoder.JSONDecodeError:
            return False 
        
    def check(self, req):
        url = 'https://www.worldquantvrc.com/account/userdata'
        check = req.post(url)
        try:
            return check.json()['status']
        except json.decoder.JSONDecodeError:
            return False

In [9]:
class CodeVariator:
    
    def __init__(self):
        self.templ_settings = '{"delay":"1","unitcheck":"off","univid":"TOP100","opcodetype":"EXPRESSION","opassetclass":"EQUITY","optrunc":0.018,"code": "%s","region":"EUR","opneut":"market","IntradayType":null,"tags":"equity","decay":10,"dataviz":"0","backdays":256,"simtime":"Y5"}'

        self.groups = ['country', 'assets/sales', 'debt/equity', 'revenue/receivable', 'sector', 'subindustry', 'current_ratio', 'industry']
        self.n = 0
        self.weights = [[2, 3], [-1, 0]]
        self.exprs = []
        self.args = []
        
    def args_neutralization(self, expr):
        template = r'a = indneutralize(a, rank(%s));\na'
        args = '['
        commas = len(self.groups) - 1
        
        exprs = []
        
        c = 0
        for group in self.groups:
            expr_1 = expr + r' = a + 0;\n' + template % group
            exprs.append(expr_1)
            alpha = self.templ_settings % expr_1
            args += alpha
            
            if c < commas:
                args += ','
            c += 1
        args += ']'
        self.args = args
        self.exprs = exprs
        return exprs
    
    def args_rank_weight(self, expr, group, prev_param, parity):
        template = r'gr1 = 10*rank(%s);\nweight1=gr1<1?%.1f:\ngr1<2?%.1f:\ngr1<3?%.1f:\
        \ngr1<4?%.1f:\ngr1<5?%.1f:\ngr1<6?%.1f:\ngr1<7?%.1f:\ngr1<8?%.1f:\ngr1<9?%.1f:\n%.1f;\na = weight1*a;\na'
        
        args = '['
        
        weights = self.weights[parity]
        
        if prev_param == -1 and parity == 1:
            parity = 0
            
        prev_weight = self.weights[0][prev_param // 10]
        size = len(weights)
        commas = size * 10 - parity * size - 1
        c = 0
        exprs = []
        for weight in weights:
            
            for i in range(10):
                a = []
                for j in range(10):
                    a.append(1)

                if parity == 1:
                    a[prev_param % 10] = prev_weight
                    if prev_param % 10 == i:
                        continue
                a[i] = weight

                expr_1 = expr + r' = a + 0;\n' + template % (group, a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8], a[9])
                exprs.append(expr_1)
                alpha = self.templ_settings % expr_1

                args += alpha 

                if c < commas:
                    args += ','
                c += 1
            
        args += ']'
        
        self.args = args
        self.exprs = exprs
        return exprs
    
    def args_group_mean(self, expr):
        template = r'b = a == a?a : 0;\n \
        a1 = a == a?scale(a) : scale( groupmean(b, %s, 1) );\na = a1;\na'
        
        args = '['
        exprs = []
        commas = len(self.groups) - 1
        c = 0
        
        for group in self.groups:
        
            expr_1 = expr + r'=a + 0;\n' + template % group
            alpha = self.templ_settings % expr_1
            args += alpha
            exprs.append(expr_1)
            
            if c < commas:
                args += ','
            c += 1
        args += ']'
        self.args = args
        self.exprs = exprs
        return exprs
            
    
        
        
            

In [12]:
class Simulator:
    def __init__(self, csrf):
        self.csrf = csrf
        self.is_simulation = False
        self.res_url_nms = []
        self.urls_corr = []
        self.n = 0
        self.results_ = []
        
        self.headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) snap Chromium/71.0.3578.98 Chrome/71.0.3578.98 Safari/537.36'}
        
    def simulate(self, args, req):
        url = 'https://www.worldquantvrc.com/simulate'
        
        sim_data = dict(args = args, _xsrf = self.csrf)
        
        simul = req.post(url, data = sim_data, headers = self.headers, verify = False)

        try:
            dct = simul.json()
            self.is_simulation = dct['status']
            if self.is_simulation:
                self.res_url_nms = dct['result']
            else:
                return dct
            return self.is_simulation
        
        except json.decoder.JSONDecodeError:
            return False
    
    def max_corr(self, corr_list):
        res_corr_lst = []
        for c in corr_list:
            if c[2] != 0:
                c[0] = float(c[0])
                c[1] = float(c[1])
                res_corr_lst.append(c)
        max_corr = max(abs(res_corr_lst[0][0]), abs(res_corr_lst[len(res_corr_lst) - 1][1]))
        return(max_corr)

    def results(self, req):
        is_done_url = 'https://www.worldquantvrc.com/job/progress/' + str(self.res_url_nms[0])
        is_done_req = req.post(is_done_url)
        while True:
            if is_done_req.text == '"DONE"':
                print('DONE')
                break
            elif is_done_req.text == '"ERROR"':
                print('Killed by\n')
                error_url = 'https://www.worldquantvrc.com/job/error/' + str(self.res_url_nms[0])
                er = req.post(error_url)
                print(er.text)
                return False
            time.sleep(25)
            print(is_done_req.text)
            is_done_req = req.post(is_done_url)
        
        sim_res = []
        
        for res_nu in self.res_url_nms:
            sim_sum_res_url = 'https://www.worldquantvrc.com/job/simsummary/' + str(res_nu)
            res_data_req = req.post(sim_sum_res_url)
            results_dct = res_data_req.json()
            
            dct = dict(sharpe = results_dct['result'][6]['Sharpe'], fitness = results_dct['result'][6]['Fitness'])
            #POSSIBLE ERROR

            sim_res.append(dct)
            
#             данные для запроса продкорреляции
            details_url = 'https://www.worldquantvrc.com/job/details/' + str(res_nu)
            hunt_for_corr = req.post(details_url)
            
            
            details_dct = hunt_for_corr.json() #POSSIBLE ERROR
            alpha_ID = details_dct['result']['clientAlphaId']
                        


            corr_url = 'https://websim2.worldquantvrc.com/correlation/start'

        #    составление запроса 
            args_for_corr_req = '{"alpha_list":["%s"],"corr_type":"prod_corr"}' %alpha_ID
            data_for_corr_req = dict(args = args_for_corr_req, _xsrf = self.csrf)
            corr_req = req.post(corr_url, data = data_for_corr_req, headers = self.headers, verify = False)

        #   URL результата
            corr_hlp_dct = corr_req.json() #POSSIBLE ERROR
            self.urls_corr.append(['https://websim2.worldquantvrc.com/correlation/result/'+str(corr_hlp_dct['result']['RequestId']), alpha_ID])
        
        corrs = []
#       цикл для выгрузки продкорреляции
        for a in self.urls_corr:
            url_of_corr_res = a[0]
            req_of_corr_res = w.post(url_of_corr_res)
            corr_res_dct = req_of_corr_res.json() #POSSIBLE ERROR
            while True:
                if corr_res_dct['result']['InProgress'] == False:
                    print("Correlation success")
                    break
                time.sleep(30)
                print(corr_res_dct['result']['InProgress'],'\n')
                req_of_corr_res = w.post(url_of_corr_res)
                corr_res_dct = req_of_corr_res.json()

            hlp1 = corr_res_dct['result']['response'].replace("true", "True")
            hlp1 = hlp1.replace("null", "None")
            hlp1_dict = eval(hlp1)

            alpha_ID = a[1]
            corr_list = hlp1_dict[alpha_ID]['Result']
            corrs.append(self.max_corr(corr_list))
        
        for i in range(len(sim_res)):
            sim_res[i]['corr'] = corrs[i]
            sim_res[i]['improve'] = i
        
        self.results_ = sim_res
        self.urls_corr = []
        return sim_res



In [10]:
import math
class AlphaHelp:
    def __init__(self, best, expr):
        self.impts = []
        self.best = best
        self.expr = expr
        self.groups = ['income/revenue', 'assets/sales', 'debt/equity', 'revenue/receivable', 'sector', 'subindustry', 'current_ratio', 'industry', 'country', 'exchange', 'operating_margin', 'current_ratio', 'quick_ratio']
        self.weights = [[2, 3], [-1, 0]]
        
    def best_alpha(self, sim_res):
        # разбиение результатов на две группы по корреляции
        hlp1 = []
        hlp03 = []
        hlp04 = []
        best = self.best
        for s in sim_res:
            if s['corr'] <= 0.4:
                hlp1.append(s)

        for s in hlp1:
            if s['corr'] <= 0.3:
                hlp03.append(s)
            else:
                hlp04.append(s)

        if hlp1 == []:
            return self.max_sharpe(sim_res)
        # выбор лучшей альфы на данном шаге
        if hlp03 == []:
            best_alpha_on_curr_step = self.max_sharpe(hlp04)[0]
            
        elif hlp04 == []:
            best_alpha_on_curr_step = self.max_sharpe(hlp03)[0]
            
        elif self.max_sharpe(hlp04)[1] - self.max_sharpe(hlp03)[1] > 0.35:
            best_alpha_on_curr_step =  self.max_sharpe(hlp04)[0]
        else:
            best_alpha_on_curr_step = self.max_sharpe(hlp03)[0]
    
        return best_alpha_on_curr_step  
    # вспомогательные функции
    def min_corr(self, lis):
        min_cor = 0
        res = []
        for item in lis:
            if item['corr'] < min_cor:
                min_cor = item['corr']
                res = item
        return [res, min_corr]

    def max_sharpe(self, lis):
        max_sh = 0
        res = []
        for item in lis:
            if abs(item['sharpe']) > max_sh:
                max_sh = abs(item['sharpe'])
                res = item
        return [res, max_sh]
    
    def compare_alphas(self, prev_res, curr_res):
        sim_res = []
        
        sim_res.append(prev_res)
        sim_res.append(curr_res)
        
        if self.best_alpha(sim_res) == prev_res:
            self.impts.append(-1)
            self.best = prev_res
            return prev_res
        else:
            self.impts.append(curr_res['improve'])
            self.best = curr_res
            return curr_res
            

In [6]:
import requests
import json
import time
csrf = '2|b90e8024|00c519c20f16c8ff6c2ca354e9be0419|1545252955'
loginer = Loginer('martiroshot@gmail.com', '5X0kwwAM', '2|b90e8024|00c519c20f16c8ff6c2ca354e9be0419|1545252955')
with requests.Session() as w:        
    a = loginer.login(w)
    print(a)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


True


In [13]:
args = '[{"delay":"1","unitcheck":"off","univid":"TOP100","opcodetype":"EXPRESSION","opassetclass":"EQUITY","optrunc":0.018,"code": "%s","region":"EUR","opneut":"market","IntradayType":null,"tags":"equity","decay":10,"dataviz":"0","backdays":256,"simtime":"Y10"}]'

expression_0 = 'ts_skewness((scale(assets)-scale(EBITDA))/scale(accounts_payable), 30)'
signal_code = r'a = %s;\na' % expression_0
args = args % expression_0


simulator = Simulator(csrf)
codeVar = CodeVariator()
simulator.simulate(args, w)
simulator.results(w)
helper = AlphaHelp(simulator.results_[0], signal_code)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


66


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


True 



/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


True 

Correlation success


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [14]:
simulator.results_

[{'corr': 0.4, 'fitness': 0.02, 'improve': 0, 'sharpe': 0.11}]

In [15]:
codeVar = CodeVariator()

In [16]:
codeVar.args_neutralization(helper.expr)
simulator.simulate(codeVar.args, w)
simulator.results(w)
helper.best = helper.best_alpha(simulator.results_)
helper.expr = codeVar.exprs[helper.best['improve']]
helper.impts.append(helper.best['improve'])

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Correlation success
True 



/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Correlation success
Correlation success
Correlation success


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [19]:
simulator.results_

[{'corr': 0.4, 'fitness': -0.0, 'improve': 0, 'sharpe': -0.03},
 {'corr': 0.4, 'fitness': 0.03, 'improve': 1, 'sharpe': 0.14},
 {'corr': 0.4, 'fitness': -0.19, 'improve': 2, 'sharpe': -0.44},
 {'corr': 0.4, 'fitness': -0.11, 'improve': 3, 'sharpe': -0.31},
 {'corr': 0.4, 'fitness': 0.04, 'improve': 4, 'sharpe': 0.15},
 {'corr': 0.4, 'fitness': -0.01, 'improve': 5, 'sharpe': -0.07},
 {'corr': 0.4, 'fitness': -0.08, 'improve': 6, 'sharpe': -0.25},
 {'corr': 0.4, 'fitness': -0.08, 'improve': 7, 'sharpe': -0.25}]

In [20]:
simulator.res_url_nms

[223285407,
 223285408,
 223285409,
 223285410,
 223285414,
 223285415,
 223285417,
 223285418]

In [21]:
simulator.urls_corr

[]

In [ ]:
codeVar.args_group_mean(helper.expr)
simulator.simulate(codeVar.args, w)
simulator.results(w)

helper.compare_alphas(helper.best, helper.best_alpha(simulator.results_))
if helper.impts[len(helper.impts) - 1] != -1:
    helper.expr = codeVar.exprs[helper.best['improve']]

In [ ]:
p = 0

In [ ]:
param = helper.impts[len(helper.impts) - 1]
exprs = []
for group in helper.groups:

    param = helper.impts[len(helper.impts) - 1]

    codeVar.args_rank_weight(helper.expr, group, param, p % 2)
    simulator.simulate(codeVar.args, w)
    simulator.results(w)
    
    exprs_prev = codeVar.exprs
    results_prev = simulator.results_
    best_prev = helper.best
    impts_prev = helper.impts
    expr_prev = helper.expr
    
    
    helper.compare_alphas(helper.best, helper.best_alpha(simulator.results_))
    p += 1
    
    print( 'step ', p, '\n', simulator.results_, '\n', helper.best )
    param = helper.impts[len(helper.impts) - 1]

    codeVar.args_rank_weight(helper.expr, group, param, p % 2)
    simulator.simulate(codeVar.args, w)
    simulator.results(w)
    
    helper.compare_alphas(helper.best, helper.best_alpha(simulator.results_))
    if helper.impts[len(helper.impts) - 1] != -1:
        helper.expr = codeVar.exprs[helper.best['improve']]
    if helper.impts[len(helper.impts) - 1] == -1 and helper.impts[len(helper.impts) - 2] != -1:
        helper.expr = exprs_prev[helper.impts[len(helper.impts) - 2]]
    p += 1
    
    print( 'step ', p, '\n', simulator.results_, '\n', helper.best )
    
    if (abs(helper.best['fitness']) > 2.0 and helper.best['corr'] <= 0.3):
        break



In [25]:
loginer.check(w)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


True

codeVar.exprs = exprs_prev
simulator.results_ = results_prev
helper.best = best_prev
helper.impts = impts_prev
helper.expr = expr_prev
p = p-1

In [14]:
helper.best

{'sharpe': 1.66, 'fitness': 1.58, 'corr': 0.3, 'improve': 15}

In [27]:
p

7